In [1]:
import os
import shutil
import pickle
import numpy as np
import torch
import copy
import matplotlib.pyplot as plt

from skopt.space import Real, Categorical, Integer

from bcnf.gp_minimize.gp_minimize import gp_minimize_fixed
from bcnf.simulation.physics import get_data
from bcnf.models.cnf import CondRealNVP
from bcnf.models.feature_network import FullyConnectedFeatureNetwork
from bcnf.eval.crossvalidate import cross_validate
from bcnf.errors import TrainingDivergedError
from bcnf.utils import get_dir

from bcnf.simulation.sampling import generate_data

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Setup

In [2]:
# HACK: scikit-optimize is not maintained anymore and this is a quick fix to make it work
# https://github.com/scikit-optimize/scikit-optimize/issues/1171#:~:text=To%20avoid%20this%20error%20in%20existing%20code%2C%20use%20int%20by%20itself.
np.int = np.int64

In [3]:
# Store and load the progress in the Huggingface repository
checkpoint_file = os.path.join(get_dir("models", "bcnf-models", "hyperparameter_optimization", "stage_2", create=True), 'checkpoint_improved.pkl')
metrics_dir = get_dir("models", "bcnf-models", "hyperparameter_optimization", "stage_2", "metrics", create=True)

print(os.path.abspath(checkpoint_file))
print(os.path.abspath(metrics_dir))

/home/psaegert/Projects/bcnf/models/bcnf-models/hyperparameter_optimization/stage_2/checkpoint_improved.pkl
/home/psaegert/Projects/bcnf/models/bcnf-models/hyperparameter_optimization/stage_2/metrics


## Data

In [4]:
param_names = ['x0_x','x0_y','x0_z','v0_x','v0_y','v0_z','g','w_x','w_y','w_z','b','A','Cd','rho','m','a_x','a_y','a_z','r']

In [5]:
# dataset_name = "hyperparameter_optimization_trajectories"

# if not os.path.exists(os.path.join(get_dir('data', 'bcnf-data'), dataset_name + '.pkl')):
#     data = generate_data(
#         n=2000,
#         type="trajectory",
#         SPF=1/30,
#         T=3,
#         config_file=os.path.join(get_dir("configs"), "throw_upwards.yaml"),
#         verbose=True,
#         break_on_impact=False,
#         name=dataset_name
#     )
# else:
#     with open(os.path.join(get_dir('data', 'bcnf-data'), dataset_name + '.pkl'), 'rb') as f:
#         data = pickle.load(f)

In [6]:
# X = np.array(data['traj'])
# y = np.column_stack([np.array(data[param]) for param in param_names])


# X_tensor = torch.Tensor(X.reshape(X.shape[0], -1))
# y_tensor = torch.Tensor(y)

# X_tensor.shape, y_tensor.shape

## Hyperparameter

In [ ]:
X, y = get_data(
    T=1.0,
    dt=1 / 30,
    N=2_000,
    break_on_impact=False
)

X_tensor = torch.Tensor(X.reshape(X.shape[0], -1))
y_tensor = torch.Tensor(y)

X_tensor.shape, y_tensor.shape

100%|██████████| 2000/2000 [00:02<00:00, 904.24it/s]


(torch.Size([2000, 90]), torch.Size([2000, 17]))

In [8]:
model_size = y_tensor.shape[1]
feature_size = X_tensor.shape[1]

print(f"Model size: {model_size}")
print(f"Feature size: {feature_size}")

Model size: 17
Feature size: 90


In [9]:
# These parameters have worked well in pilot experiments
optimizer_kwargs = {
    "lr": 2e-4
}

lr_scheduler_kwargs = {
    "mode": "min",
    "factor": 0.5,
    "patience": 250,
    "threshold_mode": "abs",
    "threshold": 1e-1,
}

In [10]:
# Define the search space
search_spaces = {
    'condition_size': Integer(1, 2048),
    'model_nested_size': Integer(16, 1024),
    'model_nested_layers': Integer(1, 8),
    'model_n_blocks': Integer(4, 32),
    'model_act_norm': Categorical([True, False]),
    'model_dropout': Real(0.0, 0.5),
    'feature_network_hidden_size': Integer(16, 256),
    'feature_network_hidden_layers': Integer(0, 16),
    'feature_network_dropout': Real(0.0, 0.5),
}

## Helper Functions

In [11]:
def param_index(name: str, search_spaces: dict[str, Real | Integer | Categorical]):
    """
    Get the index of a parameter in the search space to match the order of the parameters in the optimization function.
    
    Parameters
    ----------
    name : str
        The name of the parameter.
    search_spaces : dict[str, Real | Integer | Categorical]
        The search space.

    Returns
    -------
    int
        The index of the parameter in the search space.
    """
    return list(search_spaces.keys()).index(name)

In [12]:
def score_parameters(params: list):
    print({k: v for k, v in zip(search_spaces.keys(), params)})

    # Catch training errors (such as divergence) to filter out bad parameter sets and speed up the optimization
    try:
        # Use cross-validation to estimate the performance of the model
        fold_metrics = cross_validate(
            model_class=CondRealNVP,
            model_kwargs={
                "size": model_size,
                "nested_sizes": [params[param_index('model_nested_size', search_spaces)]] * params[param_index('model_nested_layers', search_spaces)],
                "n_blocks": params[param_index('model_n_blocks', search_spaces)],
                "n_conditions": params[param_index('condition_size', search_spaces)],
                "act_norm": params[param_index('model_act_norm', search_spaces)],
                "dropout": params[param_index('model_dropout', search_spaces)],
            },
            feature_network_class=FullyConnectedFeatureNetwork,
            feature_network_kwargs={
                "sizes": [feature_size]
                    + [params[param_index('feature_network_hidden_size', search_spaces)]] * params[param_index('feature_network_hidden_layers', search_spaces)]
                    + [params[param_index('condition_size', search_spaces)]],
                "dropout": params[param_index('feature_network_dropout', search_spaces)],    
            },
            optimizer_class=torch.optim.Adam,
            optimizer_kwargs=optimizer_kwargs,
            lr_scheduler_class=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs=lr_scheduler_kwargs,
            X=X_tensor,
            y=y_tensor,
            n_epochs=50_000,
            val_loss_patience=500,
            val_loss_tolerance=1e-1,  # Improvements to treat as significant
            val_loss_tolerance_mode="abs",
            timeout=60 * 60,  # 1 hour
            batch_size=256,
            device=device,
            verbose=True,  # Print the progress
            n_splits=3,
            errors="raise"  # Raise errors to stop the optimization
        )

        # Save the loss_history and metrics for analysis
        with open(os.path.join(metrics_dir, f'params_{"_".join([str(p) for p in params])}.pkl'), 'wb') as f:
            pickle.dump(fold_metrics, f)

    except TrainingDivergedError as e:
        print(e)
        return 100  # A big number (bad score) to avoid this parameter set

    # Print the average validation loss and its standard deviation during optimization
    val_loss_list = [r['val_loss'][1] for r in fold_metrics]  # each val_loss value is a tuple (epoch, loss)
    print(f'Val Loss: {np.mean(val_loss_list):.4f} ± {np.std(val_loss_list):.4f}')

    # Return the upper confidence bound of the validation loss
    # This encourages the optimization to find good AND reliable parameter sets
    return np.mean(val_loss_list) + np.std(val_loss_list)

In [13]:
def save_checkpoint(result):
    """
    Save the result of the optimization to a checkpoint file.
    Used as a callback in the optimization function.
    
    Parameters
    ----------
    result : OptimizeResult
        The result of the optimization.
    """
    # Safely write the result to a temporary file first without overwriting the checkpoint file
    with open(checkpoint_file + ".tmp", 'wb') as f:
        # Ignore
        # - result['specs']['args']['func']
        # - result['specs']['args']['callback']
        # because it causes problems when reading somewhere else
        result_no_func = copy.deepcopy(result)
        del result_no_func['specs']['args']['func']
        del result_no_func['specs']['args']['callback']
        pickle.dump(result_no_func, f)

    # Delete the old checkpoint file and rename the temporary file
    shutil.move(checkpoint_file + ".tmp", checkpoint_file)

## Optimization

In [14]:
# Numer of random initial points to explore the search space
N_STEPS_INIT = 30

# Number of iterations to run the optimization in total
N_STEPS = 100

In [15]:
# Load the checkpoint if it exists
if os.path.exists(checkpoint_file):
    print(f'Loading checkpoint from {checkpoint_file}')

    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)

        # Re-assign the function and callback because they are not picklable
        checkpoint['specs']['args']['func'] = score_parameters
        checkpoint['specs']['args']['callback'] = save_checkpoint

    print(f'Resuming from iteration {len(checkpoint.x_iters)}')

    n_initial_points = max(0, N_STEPS_INIT - len(checkpoint.x_iters))
    n_calls_remaining = max(0, N_STEPS - len(checkpoint.x_iters))
    x0 = checkpoint.x_iters
    y0 = checkpoint.func_vals
else:
    print('No checkpoint found. Starting new optimization')
    checkpoint = None

    n_initial_points = N_STEPS_INIT
    n_calls_remaining = N_STEPS

    x0 = None
    y0 = None
    
print(f'Running with {n_initial_points} initial points and {n_calls_remaining} remaining iterations')

No checkpoint found. Starting new optimization
Running with 30 initial points and 100 remaining iterations


In [16]:
# You might want to adjust the n_calls or other parameters based on the checkpoint
result = gp_minimize_fixed(
    func=score_parameters,
    dimensions=search_spaces.values(),
    n_initial_points=n_initial_points,  # Number of random points before starting the optimization
    n_calls=n_calls_remaining,  # Number of iterations
    random_state=2024_03_25,
    verbose=True,
    acq_func="EI",  # Expected Improvement https://arxiv.org/pdf/1009.5419.pdf
    initial_point_generator="halton", # https://en.wikipedia.org/wiki/Halton_sequence
    callback=save_checkpoint,
    x0=x0,
    y0=y0)

30 initial points will be randomly generated
Iteration No: 1 started. Evaluating function at random point.
{'condition_size': 1, 'model_nested_size': 16, 'model_nested_layers': 1, 'model_n_blocks': 4, 'model_act_norm': False, 'model_dropout': 0.0, 'feature_network_hidden_size': 16, 'feature_network_hidden_layers': 0, 'feature_network_dropout': 0.0}


Train: -8.1279 - Val: -7.9453 (avg: -7.9333, min: -7.8651) | lr: 6.25e-06 - Patience: 500/500:  15%|█▍        | 7449/50000 [02:53<16:32, 42.88it/s] 
Train: -11.9331 - Val: -10.6863 (avg: -10.6912, min: -10.6218) | lr: 6.25e-06 - Patience: 500/500:  13%|█▎        | 6259/50000 [02:27<17:10, 42.43it/s]
Train: -11.7297 - Val: -11.1704 (avg: -11.1647, min: -11.0659) | lr: 6.25e-06 - Patience: 500/500:  20%|█▉        | 9823/50000 [03:54<15:58, 41.93it/s]


Val Loss: -10.3427 ± 1.5979
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 556.0341
Function value obtained: -8.7448
Current minimum: -8.7448
Iteration No: 2 started. Evaluating function at random point.
{'condition_size': 1024, 'model_nested_size': 352, 'model_nested_layers': 2, 'model_n_blocks': 8, 'model_act_norm': False, 'model_dropout': 0.038461538461538464, 'feature_network_hidden_size': 30, 'feature_network_hidden_layers': 1, 'feature_network_dropout': 0.021739130434782608}


Train: -30.4528 - Val: -22.7752 (avg: -22.9189, min: -23.6763) | lr: 5.00e-05 - Patience: 500/500:   4%|▎         | 1818/50000 [01:17<34:04, 23.57it/s]
Train: -30.0309 - Val: -21.5282 (avg: -21.6643, min: -22.8815) | lr: 5.00e-05 - Patience: 500/500:   3%|▎         | 1678/50000 [01:13<35:18, 22.81it/s]
Train: -29.5956 - Val: -22.0164 (avg: -21.8390, min: -22.7756) | lr: 5.00e-05 - Patience: 500/500:   4%|▎         | 1820/50000 [01:16<33:53, 23.70it/s]


Val Loss: -23.3258 ± 0.3389
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 227.5756
Function value obtained: -22.9869
Current minimum: -22.9869
Iteration No: 3 started. Evaluating function at random point.
{'condition_size': 513, 'model_nested_size': 688, 'model_nested_layers': 4, 'model_n_blocks': 12, 'model_act_norm': False, 'model_dropout': 0.07692307692307693, 'feature_network_hidden_size': 44, 'feature_network_hidden_layers': 2, 'feature_network_dropout': 0.043478260869565216}


...

Train: -36.0369 - Val: -35.5816 (avg: -35.6581, min: -35.7197) | lr: 6.25e-06 - Patience: 500/500:  16%|█▌        | 8092/50000 [07:33<39:07, 17.85it/s]
Train: -31.8379 - Val: -27.9390 (avg: -28.3435, min: -28.6206) | lr: 5.00e-05 - Patience: 500/500:   6%|▌         | 3010/50000 [02:47<43:38, 17.95it/s]
Train: -28.3004 - Val: -23.1195 (avg: -23.6089, min: -25.1078) | lr: 1.00e-04 - Patience: 500/500:   5%|▍         | 2461/50000 [02:18<44:33, 17.78it/s]


Val Loss: -29.6802 ± 3.7938
Iteration No: 29 ended. Evaluation done at random point.
Time taken: 759.5702
Function value obtained: -25.8864
Current minimum: -57.6940
Iteration No: 30 started. Evaluating function at random point.
{'condition_size': 1472, 'model_nested_size': 700, 'model_nested_layers': 7, 'model_n_blocks': 10, 'model_act_norm': True, 'model_dropout': 0.121301775147929, 'feature_network_hidden_size': 186, 'feature_network_hidden_layers': 8, 'feature_network_dropout': 0.13137996219281664}


Train: -22.7504 - Val: -25.3140 (avg: -25.9832, min: -26.1537) | lr: 2.00e-04 - Patience: 3/500:   3%|▎         | 1529/50000 [02:46<1:28:04,  9.17it/s] 


Error in fold 0: Loss exploded to 569701.375 at epoch 1529.5
Loss exploded to 569701.375 at epoch 1529.5
Iteration No: 30 ended. Evaluation done at random point.
Time taken: 167.0312
Function value obtained: 100.0000
Current minimum: -57.6940
Iteration No: 31 started. Searching for the next optimal point.
{'condition_size': 1038, 'model_nested_size': 484, 'model_nested_layers': 5, 'model_n_blocks': 32, 'model_act_norm': True, 'model_dropout': 0.3185052316747453, 'feature_network_hidden_size': 97, 'feature_network_hidden_layers': 5, 'feature_network_dropout': 0.1254495493226791}


Train: -53.3390 - Val: -54.5408 (avg: -54.5381, min: -54.4880) | lr: 3.91e-07 - Patience: 500/500:  21%|██        | 10611/50000 [41:56<2:35:41,  4.22it/s]
Train: -55.4095 - Val: -50.3834 (avg: -50.4031, min: -50.3203) | lr: 7.81e-07 - Patience: 500/500:  21%|██        | 10378/50000 [41:21<2:37:53,  4.18it/s]
Train: -51.3079 - Val: -53.0534 (avg: -52.8785, min: -52.9128) | lr: 2.50e-05 - Patience: 162/500:  14%|█▍        | 7196/50000 [35:02<4:51:53,  2.44it/s]